###EJERCICIO 1 - SPARK

Dado los acontecimientos en USA, deseamos obtener datos que
nos den mayor información sobre las muertes de gente de raza negra
por parte de oficiales de policía.
Para ello, tenemos un csv con información sobre las muertes por
parte de oficiales de policía en USA para 2015 hasta 2017:
(name, date, race, city, state)
Y otro csv con información sobre el porcentaje de pobreza en las
ciudades de USA:
(state, city, poverty_rate)
Se pide:

A) Obtener las 10 ciudades con mayor diferencia entre el porcentaje
de pobreza de la ciudad y el porcentaje de pobreza del estado en el
que se encuentra esa ciudad. Por ejemplo si en la ciudad de Houston
la pobreza es de 15.2 y la pobreza en Texas (el estado donde se
encuentra Houston) es de 11.1, la diferencia es 4.1.(15 pts)

In [ ]:
#Agrego los imports necesarios
from pyspark.sql import *
from pyspark.sql.functions 
import * from pyspark 
import SparkContext from pyspark 
import SQLContext 
import random

In [ ]:
#Creo la Spark Session
spark = SparkSession.builder.getOrCreate()

#Creo el Spark Context 
sc = spark.sparkContext

#Genero mis RDDs
PobrezaRDD = sc.parallelize(pobreza) 
MuertesRDD = sc.parallelize(muertes)

In [ ]:
#(state, city, poverty_rate)
#Supongo que en el RDD hay una unica aparicion para cada city (ejemplo: (Texas, Houston, %) aparece una sola vez)
#Entonces ya tengo el % por CITY.

In [ ]:
#Pobreza = (state, (city, poverty_rate) )
Pobreza = PobrezaRDD.map(lambda x: (x[0], (x[1], x[2]) ))

In [ ]:
#Creo un RDD PobrezaEstado = (state, poverty_rate) para pode calcular el total.
PobrezaEstado = PobrezaRDD.map(lambda x: (x[0],x[2]))

In [ ]:
#Reduzo por state para obtener la suma del %  de pobreza por ciudad en ese estado.
#(state, total_poverty_rate)
PobrezaEstado = PobrezaEstado.reduceByKey(lambda a,b: a+b)

In [ ]:
#Tengo Pobreza = (state, (city, poverty_rate)) y PobrezaEstado = (state, (total_poverty_rate))

#Hago un Join por state (state, (city, city_poverty_rate, total_poverty_rate) )
PobrezaEstadoCiudad = Pobreza.join(PobrezaEstado).cache()

In [ ]:
#Mapeo mi RDD PobrezaEstadoCiudad (state, city, diferencia_poverty)
PobrezaEstadoCiudad = PobrezaEstadoCiudad.map(lambda x: (x[0], x[1][0], x[1][1]- x[1][2]) )

In [ ]:
#Me quedo con las 10 ciudades con mayor diferencia.
Top10MasDiferencia = PobrezaEstadoCiudad.takeOrdered(10, lambda x: -x[2])

B) Obtener la cantidad de muertes de gente de raza negra por parte
de oficiales de policía, agrupada por estados que compartan el mismo nivel de pobreza redondeado al entero más cercano. 
Por ejemplo, si NJ tiene una pobreza de 10.33, AL una de 20.64 y AZ una de 10.44, NJ y AZ quedarían juntos representados por el nivel de pobreza de 10 y AL en otro grupo con el nivel 21. La salida debe tener el formato:
(nivel_de_pobreza, total_de_muertes) (15 pts)

In [ ]:
#Organizo mi RDD Muertes para contar la cantidad de muertos negros por estado.
#(name, date, race, city, state) 
#(state, 1 si es negro 0 si no lo es )
Muertes = MuertesRDD.map(lambda x: (x[4] , 1 if x[2] == 'black' else 0 ))

In [ ]:
#Cuento la cantidad de muertes de negro por estado
#(state, total_negros_muertos)
MuertesNegros = Muertes.reduceByKey(lambda a,b: a+b)

In [ ]:
#Voy a utlizar el RDD hecho en el item anterior PobrezaEstado = (state, poverty_total)
#Ahora tengo MuertesNegros = (state, total_negros_muertos)

#Junto ambos RDD (state, (poverty_total, total_negros_muertos))
MuertesyPobreza = PobrezaEstado.join(MuertesNegros)

In [ ]:
#Organizo mi RDD para redondear el % y eliminar el state.
#(round_poverty_rate, total_negros_muertos)
MuertesyPobreza = MuertesyPobreza.map(lambda x: (round(x[1][0]), x[1][1]))

In [ ]:
#Agrupo por % de poverty_rate y sumo la cantidad de muertes de negros para ese %
MuertesyPobreza = MuertesyPoberza.reduceByKey(lambda a,b: a+b)